Emerging Technologies Project

Cathal Butler | G00346889

In [19]:
import keras as kr
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Ref: 
# https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/index.html?index=..%2F..index#0
# https://www.tensorflow.org/guide/data_performance
# https://codelabs.developers.google.com/codelabs/keras-flowers-data/#3


In [20]:
# The number of training examples in one forward/backward pass. 
# The higher the batch size, the more memory space you'll need.
BATCH_SIZE = 128
# Setting the number of forward passes and backward passes of all the training examples
EPOCHS = 10
# https://www.tensorflow.org/guide/data_performance
num_parallel_calls = tf.data.experimental.AUTOTUNE
AUTO = tf.data.experimental.AUTOTUNE

# Dataset
training_images_file = 'mnist_dataset/train-images-idx3-ubyte'
training_labels_file = 'mnist_dataset/train-labels-idx1-ubyte'
validation_images_file = 'mnist_dataset/t10k-images-idx3-ubyte'
validation_labels_file = 'mnist_dataset/t10k-labels-idx1-ubyte'

plt.rcParams['figure.figsize'] = (10,10)
  

In [22]:
# Function for reading in and decode labels
def read_label(tf_bytestring):
    label = tf.decode_raw(tf_bytestring, tf.uint8)
    label = tf.reshape(label, [])
    label = tf.one_hot(label, 10)
    return label


# Function to read in, decode and resize the images to 28x28
def read_image(tf_bytestring):
    image = tf.decode_raw(tf_bytestring, tf.uint8)
    image = tf.cast(image, tf.float32)/256.0
    image = tf.reshape(image, [28*28])
    return image


# Function to read both the image file and label file.
def load_dataset(image_file, label_file):
    # Images are read in with fixed length
    imagedataset = tf.data.FixedLengthRecordDataset(image_file, 28*28, header_bytes=16)
    # map is used to obtain a dataset of the images
    imagedataset = imagedataset.map(read_image, num_parallel_calls)
    # Labels are read in with fixed length
    labelsdataset = tf.data.FixedLengthRecordDataset(label_file, 1, header_bytes=8)
    # map is used to obtain a dataset of the labels
    labelsdataset = labelsdataset.map(read_label, num_parallel_calls)
    # zip is used to pain images to labels
    dataset = tf.data.Dataset.zip((imagedataset, labelsdataset))
    return dataset 

def get_training_dataset(image_file, label_file, batch_size):
    # Load in image & lable file
    dataset = load_dataset(image_file, label_file)
     # this small dataset can be entirely cached in RAM, for TPU this is important to get good performance from such a small dataset
    dataset = dataset.cache() 
    dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
    # Mandatory for Keras for now
    dataset = dataset.repeat() 
    # drop_remainder is important on TPU, batch size must be fixed
    dataset = dataset.batch(batch_size, drop_remainder=True) 
    # fetch next batches while training on the current one (-1: autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)  
    
    return dataset
  
def get_validation_dataset(image_file, label_file):
    # Load image & label file
    dataset = load_dataset(image_file, label_file)
    # this small dataset can be entirely cached in RAM, for TPU this is important to get good performance from such a small dataset
    dataset = dataset.cache() 
    # 10000 items in eval dataset, all in one batch
    dataset = dataset.batch(10000, drop_remainder=True) 
    # Mandatory for Keras for now
    dataset = dataset.repeat() 
    
    return dataset

# instantiate the datasets
training_dataset = get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_dataset = get_validation_dataset(validation_images_file, validation_labels_file)

# For TPU, we will need a function that returns the dataset
training_input_fn = lambda: get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_input_fn = lambda: get_validation_dataset(validation_images_file, validation_labels_file)

In [23]:
model = tf.keras.Sequential(
  [
        tf.keras.layers.Input(shape=(28*28,)),
        #tf.keras.layers.Dense(200, activation='relu'),
        #tf.keras.layers.Dense(60, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
  ])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [24]:
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
print("Steps per epoch: ", steps_per_epoch)

history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1)

Steps per epoch:  468
Epoch 1/10
468/468 [==============================] - 2s 4ms/step - loss: 0.6845 - acc: 0.8302 - val_loss: 0.3839 - val_acc: 0.9005
Epoch 2/10
468/468 [==============================] - 2s 5ms/step - loss: 0.3643 - acc: 0.9019 - val_loss: 0.3202 - val_acc: 0.9123
Epoch 3/10
468/468 [==============================] - 2s 4ms/step - loss: 0.3207 - acc: 0.9119 - val_loss: 0.2981 - val_acc: 0.9183
Epoch 4/10
468/468 [==============================] - 1s 2ms/step - loss: 0.3005 - acc: 0.9170 - val_loss: 0.2861 - val_acc: 0.9190
Epoch 5/10
468/468 [==============================] - 2s 3ms/step - loss: 0.2882 - acc: 0.9198 - val_loss: 0.2779 - val_acc: 0.9229
Epoch 6/10
468/468 [==============================] - 3s 6ms/step - loss: 0.2798 - acc: 0.9218 - val_loss: 0.2743 - val_acc: 0.9234
Epoch 7/10
468/468 [==============================] - 3s 6ms/step - loss: 0.2743 - acc: 0.9239 - val_loss: 0.2706 - val_acc: 0.9231
Epoch 8/10
468/468 [==============================] - 